In [1]:
import hashlib
import json
import math
import os
import random
# You may NOT alter the import list!!!!
import sys
print (sys.getrecursionlimit())

3000


In [17]:
class CryptoProject(object):

    def __init__(self):
        # TODO: Change this to YOUR Georgia Tech student ID!!!
        # Note that this is NOT your 9-digit Georgia Tech ID
        self.student_id = 'bdornier3' #'dgoodrick3'

    def get_student_id_hash(self):
        return hashlib.sha224(self.student_id.encode('UTF-8')).hexdigest()

    def get_all_data_from_json(self, filename):
        data = None
#         base_dir = os.path.abspath(os.path.dirname(__file__))
#         with open(os.path.join(base_dir, filename), 'r') as f:
#             data = json.load(f)
#         return data
        base_dir = os.getcwd()
        with open(os.path.join(base_dir, filename), 'r') as f:
            data = json.load(f)
        return data

    def get_data_from_json_for_student(self, filename):
        data = self.get_all_data_from_json(filename)
        name = self.get_student_id_hash()
        if name not in data:
            print(self.student_id + ' not in file ' + filename)
            return None
        else:
            return data[name]

    # TODO: OPTIONAL - Add helper functions below
    # BEGIN HELPER FUNCTIONS
        
    def extendedEuclidian(self, e, tot_n):
        #https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Python
        if e == 0:
            #Stop once the remainder of big%small is 0
            #big is the GCD
            return (tot_n, 0, 1)
        else:
            #break down the multiples by dividing the previous big number by it's remainder
            gcd, k, a = self.extendedEuclidian(tot_n % e, e)
            # substitute back in k and multiples of N to get d=a-k(N//rem)
            return gcd, a - k*(tot_n // e), k
        
    def mod_inv(self,e, tot_n):
        #https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Python
        gcd,raw,_ = self.extendedEuclidian(e, tot_n)
        if gcd==1:  #GCD must be one or there is no solution
            return raw%tot_n #ensures the result is positive
        return 'NaN'
    
    def binarySearch(self,val):
        # https://rosettacode.org/wiki/Binary_search#Python:_Iterative
        lower=0
        upper = val
        while lower < upper:
            mid = (lower+upper)//2
            if mid**3 < val:
                lower = mid+1
            else:
                upper = mid
        return lower
    
    # END HELPER FUNCTIONS

    def decrypt_message(self, N, e, d, c):
        # https://en.wikipedia.org/wiki/RSA_(cryptosystem)
        return hex(pow(c,d,N))

    def crack_password_hash(self, password_hash, weak_password_list):
        password = 'abc'
        salt = '123'
        for password in weak_password_list:
            for salt in weak_password_list:
                hashed_password = hashlib.sha256(password.encode() + salt.encode()).hexdigest()
                if hashed_password == password_hash:
                    print("PWD:", password, "Salt:", salt)
                    return password, salt
        
        return "not found"

    def get_factors(self, n):
        #check all odd values > sqrt(n) because p or q must be > Sqrt(n)  
        start = 2*(n**.5//2)-1
        for i in range(int(start),n,2):
            if n%i == 0:
                return i, n//i
        return 'NaN', 'NaN'

    def get_private_key_from_p_q_e(self, p, q, e):
        # TODO: Implement this function for Task 3
        # e*d≡1 mod ϕ(N) 
        # ϕ(N)=ϕ(p)ϕ(q)=(p−1)(q−1)
        # extendedEuclidian takes ϕ(N) and e and returns d such that ed+kϕ(N)=1  
        tot = (p-1)*(q-1)
        return self.mod_inv(e, tot)
        
    def is_waldo(self, n1, n2):
        # TODO: Implement this function for Task 4
        '''The greatest common divisor (GCD) of two 1024-bit integers
            can be computed in microseconds. Find two distinct RSA moduli 
            N1 and N2 that share a prime factor...'''
        if math.gcd(n1,n2) != 1:
            return True
        else:
            return False

    def get_private_key_from_n1_n2_e(self, n1, n2, e):
        # TODO: Implement this function for Task 4
        # assumption: n1 and n2 share a common factor
        # The GCD of N1 and N2 is p        
        p = math.gcd(n1, n2)
        # n2/p = q2 but we only want the private key, d, of n1 so q2 doesn't matter 
        q = n1//p
        tot = (p-1)*(q-1)
        #private key d satisfies de = 1 mod (p-1)(q-1):
        return self.mod_inv(e, tot )
        

    def recover_msg(self, N1, N2, N3, C1, C2, C3):
        # TODO: Implement this function for Task 5
        C = [C1, C2, C3]
        N = [N1,N2, N3]
        #find the n that doesn't include Ni
        n = [N2*N3,N1*N3,N1*N2]# I could have used [pN/N1, pN/N2, pN/N3]
        x = [self.mod_inv(n[i],N[i]) for i in range(len(C))]
        c = [n[i]*C[i]*x[i] for i in range(len(C))]
        s = sum(c)
        pN =N1*N2*N3
        ans = s%pN
        return self.binarySearch(ans)       

    def task_1(self):
        data = self.get_data_from_json_for_student('keys4student_task_1.json')
        N = int(data['N'], 16)
        e = int(data['e'], 16)
        d = int(data['d'], 16)
        c = int(data['c'], 16)
        self.data = data
        m = self.decrypt_message(N, e, d, c)
        return m

    def task_2(self):
        data = self.get_data_from_json_for_student('hashes4student_task_2.json')
        password_hash = data['password_hash']
        self.data = data

        # The password file is loaded as a convenience
        weak_password_list = []
#         base_dir = os.path.abspath(os.path.dirname(__file__))
        base_dir = os.getcwd()
        with open(os.path.join(base_dir, 'top_passwords.txt'), 'r', encoding='UTF-8-SIG') as f:
            pw = f.readline()
            while pw:
                weak_password_list.append(pw.strip('\n'))
                pw = f.readline()
        self.ph = password_hash
        self.wp = weak_password_list
        password, salt = self.crack_password_hash(password_hash, weak_password_list)

        return password, salt

    def task_3(self):
        data = self.get_data_from_json_for_student('keys4student_task_3.json')
        n = int(data['N'], 16)
        e = int(data['e'], 16)

        p, q = self.get_factors(n)
        d = self.get_private_key_from_p_q_e(p, q, e)

        return hex(d).rstrip('L')

    def task_4(self):
        all_data = self.get_all_data_from_json('keys4student_task_4.json')
        student_data = self.get_data_from_json_for_student('keys4student_task_4.json')
        n1 = int(student_data['N'], 16)
        e = int(student_data['e'], 16)

        d = 0
        waldo = 'Dolores'

        for classmate in all_data:
            if classmate == self.get_student_id_hash():
                continue
            n2 = int(all_data[classmate]['N'], 16)

            if self.is_waldo(n1, n2):
                waldo = classmate
                print(bin(n2))
                d = self.get_private_key_from_n1_n2_e(n1, n2, e)
                break

        return hex(d).rstrip("L"), waldo

    def task_5(self):
        data = self.get_data_from_json_for_student('keys4student_task_5.json')
        N1 = int(data['N0'], 16)
        N2 = int(data['N1'], 16)
        N3 = int(data['N2'], 16)
        C1 = int(data['C0'], 16)
        C2 = int(data['C1'], 16)
        C3 = int(data['C2'], 16)
        
        m = self.recover_msg(N1, N2, N3, C1, C2, C3)
        # Convert the int to a message string
        msg = bytes.fromhex(hex(m).rstrip('L')[2:]).decode('UTF-8')

        return msg

cp = CryptoProject()
# m = cp.task_1() # not working
print(m)
# cp.task_2()
# cp.task_3() #not working
cp.task_4()
# cp.task_5()

0x9d20b02c627f60f82f2fe07b956c3ca
0b1101011010101110011010111100010100101001010000001100011100000100000100110001110111011101001101011011100101101110100110010000110100000010111000110111110011101010000001010111101110100100100111101010010101101001000011101110000000100001000001011110000101111110110010001110011101111011010101001101101111100010110111110010100011011011001101110101011111010011010111101000000101101101111101001010010101110010010000110011011101111110111001001000110000111011110000010001101111011111110011100010001100111011111111000000010010000000101011110010010010010011111011000001101110111101101100110000101111000000001011001001110100101110110000110001100010111101011101111000110110101001110001100010000100011010100111101111110000101111010110101001000110000000101010111001011110001100001011011111101011101111110000110010001001110111000111111110101101111110000000011010101010001001010110101000101111110101011111111000011001010111100011011100011000101011111100101111100001010101010010101010

('0x8d117f2e5deb5f09b599f8732df4ec8f81d1af838e3f3e859235b34254c5d8b44de982d743e885f24e3a190cc1876678d4929d32d4bcc7af683a26f6b857bd2658229570324e40d950235d9f2265ef4b90744ff8b9fb7e502857bbae7a30d75743cad23b1d5bd96109a26094116c2013c5db92d2c715aeee5208f093392f2fea',
 'be729fbeb59825c8cc22fe7255b71e5123f320cdb65987a197884b98')

In [10]:
classmate = 0xbf99be2fa248e1165ee56977b87edff43fdebeaeb2a40e615529b5900d890d1f35b97b565a455a7c41625c6d164c97d1201421af96fdedb6aafbf0bc2eb02b9449c4d6f1ea0312bba156e93b0ffc943263632b9c64ef0317b5171aecbf5b2a8f86b4e122d78238a688702dbac2400973558b40e1f1d269393b16b444380ca339
result =    0xb38a5359b6bc7f29a5ea53f6100a214d74f17bf7d5028555cf22bfea02014c2f5bc09abec1e1ada08bb725bf2911ab5e24a94eb0f43b7c1e84ddfa61637670e61e644b7ea626ccfcf5c099671bb58916e4a89f8549a8a416f480b69b1a113882f01f7b8c736ebbf407d2fd6e24c43cd3bc461e7840254ebe5789d7e25966ff61

3

In [24]:
# e = 3
# N1 = 629
# N2 = 2173
# N3 = 1159

# C1 = 529
# C2 = 414
# C3 = 558
#find c = C1 mod N1 and C2 mod N2 and C3 mod N3
    #C1 part   #c2 part   C3 part
x = N2*N3     + N1*N3    + N1*N2
print(x%C1, x%C2, x%C3)
#C1 = m^3 mod N1 => C1%N1 = M^3
print (x)
# N2*N3*?*C1 = C1%N1
# N1*N3*?*C2 = C2%N2
# N1*N2*?*C3 = C3%N3
2173+1159+559+529+629+1159+1603+414+629+2173+433+558
# 629*2173*1159
15625%(629*2173*1159)
15625**(1/3.)

63565808541420448412430171070546645478172386150469462666668095536137015904412530106641137012372017537979886925528387588929766525471360619743458548080484492007256100865858604086079351466395089834165121334424462243264705910681062354228078574379032911081312479121820094335798164909243317424541035613941625366971 70918679732377196019344620845676823322168534514770607168970605761957101256767669419281024102348237680437998991210648158102105831266069141892759852335213840930332905751163226349596999146056408612079294811947516145272958750714633968390148042454851667743166956582520732813365385414324863968320647903549867569589 11940315421582380814173394175512118320424158588800159514394213541181972018691721370490773257224059438234561720552742984703130973709953159849110129949652385437789479783322335977555230341856965277211576631181115829259769208412614052479352221755336223324913293099154076208577521349807110457236148262787314489169
5647348586411649993349894103102107202540340751853155188686934124496283301

24.999999999999996

In [60]:
cp.mod_inv(20,3)


2

In [18]:
# c* = ci mod (Ni)
import numpy as np
C = [C1, C2, C3]
N = [629,2173, 1159]
pN = np.product(N)
print(pN)
n = [pN/N[i] for i in range(len(C))]
print(n)
x = [cp.mod_inv(n[i],N[i]) for i in range(len(C))]
print (x)
c = [n[i]*C[i]*x[i] for i in range(len(C))]
print(c)
s = np.sum(c)
print(s)
ans = s%pN
print(ans)
m = ans**(1/3.)
print (int(m+1))

1584140903
[2518507.0, 729011.0, 1366817.0]
[559.0, 1603.0, 433.0]
[744750223477.0, 483802318062.0, 330242122638.0]
1558794664177.0
15625.0
25


136930068973653733934777066324453098441146702243411312246442286371085687833685685644990844995326379848961733762072006113203332180727975513509810778191132458974105958777668744713114847383016755797085057088340136249718838574139044758097976389689133689368223161868166930224424740213330847377444126662076830742520